In [1]:
# MIT image : Analysis
# Made by : HERO Kwon
# Date : 190113
from numba import jit
import os
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
import time

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'
#path_mit_image = '/home/herokwon/mount_data/Data/Wi-Fi_mit_image/'

path_csi = 'D:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'D:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'D:\\Data\\Wi-Fi_meta\\'
path_sc = 'D:\\Data\\Wi-Fi_info\\'
path_mit_image = 'D:\\Data\\Wi-Fi_mit_denoise_debug\\'
path_movie = 'D:\\Data\\Wi-Fi_movie\\'

# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv')
#df_info = df_info[df_info.id_location==1 ]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 100 #15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')

# 3D scan param
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 8#3
max_subc = 30

# Load data
saved_images = [f.replace(".npy","") for f in os.listdir(path_mit_image)]
saved_images

df11 = df_info[(df_info.id_direction==1) & (df_info.id_location==1)]
df_target = df11[df11['id'].isin(saved_images)]
df_lab = df_target[['id','id_person','id_location','id_direction','id_exp']]
df_lab = df_lab.drop_duplicates()
df_lab

data_mit = []
filename_mit = []
for file in df_lab.id.values:
    data_load = np.load(path_mit_image + file + '.npy')
    #dl_norm = np.array([data_load[i] / np.max(data_load[i]) for i in range(data_load.shape[0])])
    dl_norm = data_load
    data_mit.append(dl_norm)
    filename_mit.append(file)

In [2]:
data_load.shape

(100, 10, 10)

In [3]:
label_mit = df_lab.id_person.values
file_mit = df_lab.id.values
label_mit

array([ 18,  18,  18,  18,  18,  18,  18,  18,  18,  18,  18,  19,  19,
        19,  19,  19,  19,  19,  19,  19,  19,  22,  22,  22,  22,  22,
        22,  22,  22,  22,  22,  22,  24,  24,  24,  24,  24,  24,  24,
        24,  24,  24,  24,  24,  25,  25,  25,  25,  25,  25,  25,  25,
        25,  25,  25,  25,  26,  26,  26,  26,  26,  26,  26,  27,  27,
        27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  28,  28,  28,
        28,  28,  28,  28,  28,  28,  28,  28,  29,  29,  29,  29,  29,
        29,  29,  29,  29,  29,  29,  29,  29,  30,  30,  30,  30,  30,
        30,  30,  30,  30,  30,  30,  30,  31,  31,  31,  31,  31,  31,
        31,  31,  31,  31,  32,  32,  32,  32,  32,  32,  32,  32,  32,
        32,  32,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,  33,
        33,  33,  34,  34,  34,  34,  34,  34,  34,  34,  34,  34,  34,
        35,  35,  35,  35,  35,  35,  35,  35,  35,  35,  35,  35,  36,
        36,  36,  36,  36,  36,  36,  36,  36,  36,  36,  36,  3

In [4]:
arr_mit = np.nan_to_num(np.array(data_mit))
#diff_mit = np.diff(arr_mit,axis=1)
sum_mit = np.sum(arr_mit,axis=1).reshape([-1,10*10])
arr_mit.shape

(858, 100, 10, 10)

In [5]:
a1 = arr_mit.reshape([-1,10*10*100])

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100,whiten=True)
pca.fit(a1)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [7]:
#plotting PCA
from matplotlib.pyplot import cm
X_pca = pca.transform(a1)
target_ids = np.unique(label_mit)
color=cm.rainbow(target_ids)
#color=cm.tab20(np.linspace(0,1,13))
'''
from matplotlib import pyplot as plt
plt.figure()
for i, c, label in zip(target_ids, color, target_ids):
    plt.scatter(X_pca[label_mit==label, 0], X_pca[label_mit==label, 1],c=c, label=label)
plt.legend()
plt.show()
'''

'\nfrom matplotlib import pyplot as plt\nplt.figure()\nfor i, c, label in zip(target_ids, color, target_ids):\n    plt.scatter(X_pca[label_mit==label, 0], X_pca[label_mit==label, 1],c=c, label=label)\nplt.legend()\nplt.show()\n'

In [8]:
X_pca.shape

(858, 100)

In [9]:
#arr_mit.reshape([-1,100*10*10]).shape

In [10]:
arr_mit.shape

(858, 100, 10, 10)

In [11]:
norm_mit = X_pca
#norm_mit = np.array([sum_mit[i,:] / np.max(sum_mit[i]) for i in range(len(sum_mit))])

In [12]:
# Basis : RM2
#@jit
def RMmodel(order,X):
    m,l = X.shape
    M1 = []
    M2 = []
    M3 = []
    MM1 = []
    MM3 = []
    Msum = np.sum(X,axis=1)
    for i in range(order):
        for k in range(l):
            M1.append(X[:,k]**(i+1))
            if (i>0):
                M3.append(X[:,k]*Msum**(i)) 
        M2.append(Msum**(i+1))
        MM1.append(M1)
        if (i>0):
            MM3.append(M3)
    MM1 = np.array(MM1).T
    MM1 = MM1.reshape((m,-1,1)).squeeze(axis=2)
    M2 = np.array(M2).T
    if (len(MM3)):
        MM3 = np.array(MM3).T
        MM3 = MM3.reshape((m,-1,1)).squeeze(axis=2)
        P = np.concatenate((np.ones((m,1)),MM1,M2,MM3),axis=1)
    else : P = np.concatenate((np.ones((m,1)),MM1,M2),axis=1)
    return(P)

#Models : LSE,RM,TER
#@jit
def Model3(rank,r,n,X,Y):
    # LSE
    alpha_lse = []
    for k in list(set(Y)):
        P_lse = X
        I_lse = np.eye(P_lse.shape[1])
        b_lse = 10**(-4)
        y_lse = (Y==k).astype('int')
        ak_lse = np.dot(np.dot(np.linalg.pinv(b_lse*I_lse + P_lse.T.dot(P_lse)),P_lse.T),y_lse)
        alpha_lse.append(ak_lse)    
    # RM
    alpha_rm = []
    for k in list(set(Y)):
        P_rm = RMmodel(rank,X)
        I_rm = np.eye(P_rm.shape[1])
        b_rm = 10**(-4)
        y_rm = (Y==k).astype('int')
        ak_rm = np.dot(np.dot(np.linalg.pinv(b_rm*I_rm + P_rm.T.dot(P_rm)),P_rm.T),y_rm)
        alpha_rm.append(ak_rm)  
    # TER
    alpha_ter = []
    for k in list(set(Y)):
        P_n = RMmodel(rank,X[Y!=k])
        P_p = RMmodel(rank,X[Y==k])
        mk_n = X[Y!=k].shape[0]
        mk_p = X[Y==k].shape[0]
        yk_n = (r-n) * np.ones(shape=Y[Y!=k].shape)
        yk_p = (r+n) * np.ones(shape=Y[Y==k].shape)
        I = np.eye(P_n.shape[1])
        b = 10**(-4)
        first_eq = np.linalg.pinv(b*I + (1/mk_n)*(P_n.T).dot(P_n) + (1/mk_p)*(P_p.T).dot(P_p))
        second_eq = (1/mk_n)*(P_n.T).dot(yk_n) + (1/mk_p)*(P_p.T).dot(yk_p)
        ak = np.dot(first_eq,second_eq)
        alpha_ter.append(ak)
    return(np.array(alpha_lse).T, np.array(alpha_rm).T,np.array(alpha_ter).T)

In [13]:
# change label
uniq_label = np.unique(label_mit)
label_table = pd.Series(np.arange(len(uniq_label)),index=uniq_label)
label_num = np.array([label_table[num] for num in label_mit])
len(uniq_label)

81

In [14]:
# K-Fold training data
skf = StratifiedKFold(n_splits=5)
split = 0
data_v = norm_mit
data_l = label_num
df_acc = pd.DataFrame()


In [15]:
# iterate Split data
for train_index,test_index in skf.split(data_v,data_l):
    data_train,data_test = data_v[train_index],data_v[test_index]
    label_train,label_test = data_l[train_index],data_l[test_index]
    # iterate Rank 1~5    
    for j in range(1):
        rank = j+1
        #Traning Result of models
        alpha_lse, alpha_rm, alpha_ter = Model3(rank,0.5,0.5,data_train,label_train)
        #Test: LSE
        time_now = time.time()
        Pt_lse = data_test
        yt_lse = Pt_lse.dot(alpha_lse)
        yt1_lse = np.argmax(yt_lse,axis=1)
        pred_true = np.equal(label_test,yt1_lse)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','LSE',split,0,acc,time_elapsed],index=['Data','Model','Split','Rank','Acc','Time'])
        df_acc = df_acc.append(res_ser,ignore_index=True)
        #Test: RM
        time_now = time.time()
        Pt_rm = RMmodel(rank,data_test)
        yt_rm = Pt_rm.dot(alpha_rm)
        yt1_rm = np.argmax(yt_rm,axis=1)
        pred_true = np.equal(label_test,yt1_rm)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','RM',split,rank,acc,time_elapsed],index=['Data','Model','Split','Rank','Acc','Time'])
        df_acc = df_acc.append(res_ser,ignore_index=True)        
        #Test: TER
        time_now = time.time()
        Pt_ter = RMmodel(rank,data_test)
        yt_ter = Pt_ter.dot(alpha_ter)
        yt1_ter = np.argmax(yt_ter,axis=1)
        pred_true = np.equal(label_test,yt1_ter)
        acc = np.count_nonzero(pred_true) / len(pred_true)
        time_elapsed = time.time() - time_now
        res_ser = pd.Series(['Data','TER',split,rank,acc,time_elapsed],index=['Data','Model','Split','Rank','Acc','Time'])
        df_acc = df_acc.append(res_ser,ignore_index=True)
    print('Data:' + '/ Split:'+ str(split))
    split += 1

C:\Users\herok\Anaconda3\envs\mit_image\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Data:/ Split:0
Data:/ Split:1
Data:/ Split:2
Data:/ Split:3
Data:/ Split:4


In [16]:
df_acc

,Acc,Data,Model,Rank,Split,Time
0,0.023474,Data,LSE,0.0,0.0,0.0
1,0.023474,Data,RM,1.0,0.0,0.0
2,0.014085,Data,TER,1.0,0.0,0.0
3,0.010526,Data,LSE,0.0,1.0,0.0
4,0.010526,Data,RM,1.0,1.0,0.0
5,0.021053,Data,TER,1.0,1.0,0.0
6,0.012739,Data,LSE,0.0,2.0,0.0
7,0.019108,Data,RM,1.0,2.0,0.0
8,0.012739,Data,TER,1.0,2.0,0.0
9,0.000000,Data,LSE,0.0,3.0,0.0
